# Evaluating Statistical Models for Stakeholder Needs

This tutorial will focus on methods for evaluating, interpreting, and reporting on the results of models based on the needs of stakeholders. 

Throughout this tutorial, we will cover the following topics:
- overview of model evaluation
  - commonly used model metrics for regressors and classifiers
  - model metrics for individual predictors/features
- modeling purposes
  - models for explanation
  - models for prediction
- identifying and understanding stakeholder needs
  - understanding model purpose based on questions
  - selecting and evaluating models 
  - making concrete recommendations
  
This tutorial notebook will use the `diabetes` dataset available in scikit-learn's `datasets` module. Two additional notebooks are available for you to follow along using the dataset that's of most interest to you. Please use that notebook for the exercise portion of the tutorial, and use this notebook as a reference.

Let's start by importing `numpy`, `pandas`, `seaborn`, and the `diabetes` dataset.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

# Import diabetes dataset from sklearn
from sklearn import datasets
db = datasets.load_diabetes(as_frame = True)

# Extract the dataset as a pandas dataframe
df = db.frame

The `diabetes` dataset has 442 data points with 10 potential predictors of the target -- a continuous measure of the progression of the disease. This information is available in the `dataset.DESCR` description.

In [ ]:
# Explore the dataset info
print(db.DESCR)

## Explore Relationships between Variables

For this tutorial, we will primarily use linear models for ease of explanation and exploration of relationships between predictors. In order to start exploring linear relationships, we will generate a correlation matrix of the dataset.

In [ ]:
# Create a correlation matrix to identify potential predictors
df_corr = df.corr().round(2)
df_corr

If we start by looking at correlation strengths with the `target`, we see that a number of moderate to strong correlation strengths. 

For the purpose of this tutorial, we will create a subset of features that are at least moderately (commonly defined as `r>=0.3`) correlated with the target. I strongly recommend you spend additional time on your own fine tuning linear models to ensure the shape of relationships is properly represented in your model.

In [ ]:
# Setting a threshold for correlation values; optional
target_corr = df_corr["target"][(np.absolute(df_corr["target"])>0.3)]
target_corr

You can easily explore visual relationships between variables using seaborn's `pairplot`.

In [ ]:
# Explore the shape of relationships 
sns.pairplot(df[target_corr.index]);

## Fit a Model

We'll fit a linear model and evaluate it using the $r^2$ score and mean squared error.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

In [ ]:
# Set up X to only include selected column names
X = df[target_corr.index]

# Fill missing data with 0; you can change this if you choose
X = X.fillna(0)

# Drop outcome variable
X = X.drop("target", axis = "columns")

# Get y
y = df["target"]

## Split into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

## Standardize the data
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
lr = LinearRegression()

# Fit the model
lr.fit(X_train, y_train)

# Generate predictions based on the test set 
y_preds = lr.predict(X_test)

In [ ]:
# Model evaluation metrics
print(f"{lr} training set r2 score: {lr.score(X_train, y_train).round(2)}")
print(f"{lr} test set r2 score: {r2_score(y_test, y_preds).round(2)}")

print(f"{lr} test set mean squared error: {mean_squared_error(y_test, y_preds).round(2)}")

# Modeling Purposes

Chester Ismay and Albert Kim's book, [Modern Dive](https://moderndive.com/5-regression.html), provides an excellent synopsis of the two broad purposes of modeling. To summarize, a model is typically fit for one of two purposes: explanation, and prediction.

**Models for explanation** typically involve more detailed exploration of the significance/importance of individual predictors than scikit-learn makes easily available. We'll use `statsmodels` for that purpose.

**Models for prediction** focus on the predictive strength of the overall model and future predicted values of `y`. In this case, you generally do not need to understand the detailed relationships between individual predictors and variables.

In [ ]:
import statsmodels.api as sm

A constant needs to be added to your model for direct comparison with the scikit-learn output.

In [ ]:
x_train = sm.add_constant(X_train)
model = sm.OLS(y_train, x_train)
results = model.fit()

Adding a constant removes the predictor names, so you need to re-add them as follows.

In [ ]:
print(results.summary(xname = ["const"] + list(X.columns)))

Two of the predictors (s4 = thyroid hormone, and s6 = blood glucose level) have very low `t` values, and non-significant `p` values. 

In combination with the other predictors in the model, they do not add new value to the model and can likely be dropped.

## Leveraging this model for different purposes
- Modeling for explanation - some questions you might ask of this data include:
  - What single best recommendation should be made to diabetes patients to improve their outcomes?
  - A series of drugs to improve blood glucose levels will be distributed for free to diabetic patients. Will it be effective?


- Modeling for prediction: 
  - What is the expected disease progression of diabetes for a series of 1,000 newly diagnosed patients?
  
When modeling for explanation, we would likely return and perform a more sophisticated modeling of individual predictors. We will start by dropping the two predictors that are not significant and re-evaluating the model. We might also want to reduce the model threshold and consider the value of other predictors.

In [ ]:
# Modeling for explanation steps: fine tune the model
target_corr = df_corr["target"][(np.absolute(df_corr["target"])>0.2)]
X = df[target_corr.index]

# Fill missing data with 0; you can change this if you choose
X = X.fillna(0)

# Drop outcome variable and extraneous predictors
X = X.drop(["target", "s4", "s6"], axis = "columns")

## Split into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

## Standardize the data
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

x_train = sm.add_constant(X_train)
model = sm.OLS(y_train, x_train)
results = model.fit()

In [ ]:
print(results.summary(xname = ["const"] + list(X.columns)))

## Addressing Stakeholder Needs

### Explanation
#### Model Selection
In general, when using a statistical model to explain the impact of a process, you will be able to most comprehensively report on a model that provides the _strength_ and _directionality_ of the relationship between individual coefficients and the target variable. In short:
- start with linear models, or other models that allow for comprehensive reporting on relationships. The [statsmodels documentation](https://www.statsmodels.org/dev/index.html) includes many options to explore
- optimize for the _strength_ and _shape_ of relationships with key predictors and the outcome variable

#### Reporting on Results
When providing detailed explanations of a model, you will likely be providing a written summary of conclusions of the overall model that is only a partial representation of the entire set of results. I strongly recommend your explanation include the following summary:
- a list of all predictors included in the model
- the overall R-squared (amount of variation predicted overall)
- the relative _strength_ and _direction_ of the relationship between key actionable predictors and y
- a comparison of the _strength_ of the relationship between key actionable predictors and other predictors
- a visual representation of the relationship between x and y (e.g., a scatterplot)

In [ ]:
sns.regplot(x="s3", y="target", data=df, marker="+");